In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
data = pd.read_csv('data/train.csv')

In [5]:
class_label = data['class'].unique()
class_label

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화'], dtype=object)

In [6]:
#전처리
import re

def preprocess_sentence(sentence):
    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = sentence.replace("\n", "")         # 구분자
    sentence = sentence.replace("\r", "")         # 구분자
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z,가-힣,0-9, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-zA-Z가-힣0-9\.\?\!,]"," ",sentence)
    sentence = sentence.strip()
    return sentence

In [7]:
X = data['conversation'].apply(preprocess_sentence)
y = data['class']

In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [13]:
X_list = list(np.stack(X.values))


In [15]:
X_list[0]

'지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해 ? 진짜 죽여버리고 싶게 . 정말 잘못했습니다 . 너가 선택해 . 너가 죽을래 네 가족을 죽여줄까 . 죄송합니다 . 정말 잘못했습니다 . 너에게는 선택권이 없어 . 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야 . 선택 못하겠습니다 . 한번만 도와주세요 . 그냥 다 죽여버려야겠군 . 이의 없지 ? 제발 도와주세요 .'

## KoBert

In [8]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf

# ! pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [9]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

In [18]:
sample = tokenizer.encode(X_list[0])

In [17]:
tokenizer.decode(sample)

'[CLS] 지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게. 정말 잘못했습니다. 너가 선택해. 너가 죽을래 네 가족을 죽여줄까. 죄송합니다. 정말 잘못했습니다. 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야. 선택 못하겠습니다. 한번만 도와주세요. 그냥 다 죽여버려야겠군. 이의 없지? 제발 도와주세요.[SEP]'

In [19]:
inputs = tokenizer(X_list,return_tensors='tf', padding=True, truncation=True,max_length=476)


In [20]:
X = tf.multiply(inputs.input_ids,inputs.attention_mask)

In [21]:
X.numpy()

array([[   2, 4299, 1457, ...,    0,    0,    0],
       [   2, 1315, 5872, ...,    0,    0,    0],
       [   2, 1457, 1763, ...,    0,    0,    0],
       ...,
       [   2, 1375, 3088, ...,    0,    0,    0],
       [   2, 1528, 6162, ...,    0,    0,    0],
       [   2,  517, 7993, ...,    0,    0,    0]], dtype=int32)

### 데이터 분할

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y, test_size=0.2, random_state=42)


## 모델

In [23]:
X.shape[1]

476

In [24]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 128
hidden_size = 256
NUM_CLASSES = len(class_label)
MAX_LENGTH = X.shape[1]
VOCAB_SIZE = tokenizer.vocab_size

In [25]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(
    VOCAB_SIZE, embedding_dim) )
          
# 양방향 lstm. dropout 0.1 적용
model.add(Bidirectional(LSTM(hidden_size,return_sequences=True, dropout=0.1)))  
model.add(Bidirectional(LSTM(hidden_size,return_sequences=True, dropout=0.1)))  
model.add(Bidirectional(LSTM(hidden_size, dropout=0.1)))  

# 모델 복잡성을 위해 dense을 2층 쌓음 
model.add(Dense(hidden_size, activation='relu'))
model.add(Dense(int(hidden_size/2), activation='relu'))

model.add(Dense(NUM_CLASSES, activation='softmax'))  

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1024256   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 512)         788480    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 512)         1574912   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 5

In [26]:
epochs=10

history = model.fit(X_train,
                    y_train,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(X_test, y_test),
                    verbose=1)

results = model.evaluate(X_test, y_test, verbose=2)

Epoch 1/10
13/13 [==============================] - 38s 1s/step - loss: 1.3777 - accuracy: 0.2861 - val_loss: 1.2821 - val_accuracy: 0.3595
Epoch 2/10
13/13 [==============================] - 14s 1s/step - loss: 1.1132 - accuracy: 0.4582 - val_loss: 1.0609 - val_accuracy: 0.4443
Epoch 3/10
13/13 [==============================] - 14s 1s/step - loss: 0.8874 - accuracy: 0.5706 - val_loss: 0.9920 - val_accuracy: 0.5658
Epoch 4/10
13/13 [==============================] - 14s 1s/step - loss: 0.6289 - accuracy: 0.7528 - val_loss: 1.0332 - val_accuracy: 0.6215
Epoch 5/10
13/13 [==============================] - 14s 1s/step - loss: 0.4547 - accuracy: 0.8396 - val_loss: 0.8609 - val_accuracy: 0.7152
Epoch 6/10
13/13 [==============================] - 14s 1s/step - loss: 0.2971 - accuracy: 0.8968 - val_loss: 1.0300 - val_accuracy: 0.6848
Epoch 7/10
13/13 [==============================] - 15s 1s/step - loss: 0.2199 - accuracy: 0.9313 - val_loss: 1.0547 - val_accuracy: 0.7418
Epoch 8/10
13/13 [==